# Spacetime Geometry Computations

This notebook is based on the original notebook: https://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_basic_Schwarzschild.ipynb

In this notebook we will use the Schwarzschild spacetime to show the essential computations that we will need during the course. The corresponding tools have been developed within the [SageManifolds](https://sagemanifolds.obspm.fr) project.

For a given metric $g_{\mu\nu}$ we can compute:
- The inverse metric: $g^{\mu\nu}$.
- Christoffel Symbols: ${\Gamma^{\lambda}}_{\mu\nu}= \frac{1}{2}g^{\lambda\sigma}\left(\partial_\mu g_{\sigma\nu}+\partial_\nu g_{\sigma\mu}-\partial_\sigma g_{\mu\nu} \right)$.
- Riemann tensor: ${R^\lambda}_{\mu\nu\sigma}=\partial_\nu {\Gamma^\lambda}_{\mu\sigma}-\partial_\sigma {\Gamma^\lambda}_{\mu\nu}+\Gamma^\eta_{\mu\sigma}\Gamma^\lambda_{\eta\nu}-\Gamma^\eta_{\mu\nu}\Gamma^\lambda_{\eta\sigma}$.
- Ricci tensor: $R_{\mu\nu} = {R^\lambda}_{\mu\lambda\nu}$.
- Scalar curvature: $R = g^{\mu\nu}R_{\mu\nu}$.
- Einstein tensor: $G_{\mu\nu} = R_{\mu\nu}-\frac{1}{2}g_{\mu\nu}R$.

Although the notebook is created for 1+3 dimensions it can be generalized to whatever 1+n dimensional Lorentzian metric.

A more advanced notebook about Schwarzschild spacetime, involving many coordinate charts, more tensor calculus and graphical outputs, is available [here](http://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Worksheets/v1.3/SM_Schwarzschild.ipynb).

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v1.3/SM_basic_Schwarzschild.ipynb) to download the original notebook file (ipynb format). To run it, you must start SageMath with the Jupyter interface, via the command `sage -n jupyter`.

In [1]:
version() #SageMath version
%display latex #To display LaTeX expressions in some outputs

## Spacetime manifold

We define our manifold (spacetime). The method `Manifold()` must receive the following arguments: $1+n$ dimension of the spacetime; name of the manifold; `structure = 'Lorentzian'` to work with a Lorentzian manifold.

In [2]:
n=3 #space dimensions
M = Manifold(1+n, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


## List of coordinates

We must define our coordinates via the method `chart()` applied to the object `M` (our spacetime manifold). Note that the argument of `chart()` is a raw string (hence the prefix `r` in front of it), which 
defines the range of each coordinate, if different from $(-\infty, +\infty)$, as well as its LaTeX symbol, if different from the Python symbol to denote the coordinate. The Python variables for each coordinate are declared within the `<...>` operator on the left-hand side of the identity, `X` denoting the Python variable chosen for the coordinate chart.

As an example, the standard **Schwarzschild coordinates** are introduced. The coordinates are the following:
$$
    t, \quad r\in (0,+\infty), \quad \theta\in(0,\pi), \quad \phi\in(0,2\pi).
$$

In [3]:
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi")
X

Chart (M, (t, r, th, ph))

In [4]:
X[:]

(t, r, th, ph)

The coordinates follows the same indexing: $X^0=t$, $X^1=r$, $X^2=\theta$, $X^3=\phi$.

In [5]:
X[0],X[1],X[2],X[3]

(t, r, th, ph)

## Metric tensor $g_{\mu\nu}$.

We introduce first the mass parameter $m$ as a symbolic positive variable, via the function `var()`:

In [6]:
m = var('m')
assume(m>=0)

The metric tensor of the Lorentzian manifold `M` is returned by the method `metric()`; we initialize its components in the chart `X`, which is the default (unique) chart on `M`:

In [7]:
g = M.metric()
g[0,0] = -(1-2*m/r)
g[1,1] = 1/(1-2*m/r)
g[2,2] = r^2
g[3,3] = (r*sin(th))^2
g.display()

g = (2*m/r - 1) dt*dt - 1/(2*m/r - 1) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

To display the metric as a matrix:

In [8]:
g[:]

[     2*m/r - 1              0              0              0]
[             0 -1/(2*m/r - 1)              0              0]
[             0              0            r^2              0]
[             0              0              0  r^2*sin(th)^2]

In order to access to a the component of the metric with components $(\mu,\nu)$ we would write: `g[mu,nu]`. Where `mu` and `nu` are integer variables such that $\mu,\nu \in \{0,\dots,n\}$, where $\{t,r,\theta,\phi\}\equiv\{0,1,2,3\}$ for our case. Here, we display the component $g_{tt}$.

In [9]:
g[0,0]

2*m/r - 1

The inverse metric can be computed via `g.inverse()`.

In [10]:
ginv=g.inverse(); ginv

Tensor field inv_g of type (2,0) on the 4-dimensional Lorentzian manifold M

In [11]:
ginv.display()

inv_g = r/(2*m - r) d/dt*d/dt - (2*m - r)/r d/dr*d/dr + r^(-2) d/dth*d/dth + 1/(r^2*sin(th)^2) d/dph*d/dph

In [12]:
ginv[:]

[      r/(2*m - r)                 0                 0                 0]
[                0      -(2*m - r)/r                 0                 0]
[                0                 0            r^(-2)                 0]
[                0                 0                 0 1/(r^2*sin(th)^2)]

If we multiply both matrices, we should get the $(1+n)\times(1+n)$ identity matrix

In [13]:
delta = g['_{ab}']*ginv['^{bc}']

In [14]:
delta[:]

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]

## Christoffel symbols ${\Gamma^{\lambda}}_{\mu\nu}$.

The Christoffel symbols of $g$ with respect to the given coordinates are
printed by the method `christoffel_symbols_display()` applied to the metric object `g`. By 
default, only the nonzero symbols and the nonredundant ones (taking into account the symmetry of the last two indices) are displayed. Type `g.christoffel_symbols_display?` to see all possible options.

In [15]:
g.christoffel_symbols_display()

Gam^t_t,r = -m/(2*m*r - r^2) 
Gam^r_t,t = -(2*m^2 - m*r)/r^3 
Gam^r_r,r = m/(2*m*r - r^2) 
Gam^r_th,th = 2*m - r 
Gam^r_ph,ph = (2*m - r)*sin(th)^2 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

Accessing to a Christoffel symbol specified by its indices (e.g. ${\Gamma^t}_{t\,r}$):

In [16]:
g.christoffel_symbols()[0,0,1]

-m/(2*m*r - r^2)

Checking the symmetry on the last two indices:

In [17]:
g.christoffel_symbols()[0,0,1] == g.christoffel_symbols()[0,1,0]

True

## Riemann curvature tensor

The Riemann curvature tensor is obtained by the method `riemann()`:

In [18]:
Riem = g.riemann()
print(Riem)

Tensor field Riem(g) of type (1,3) on the 4-dimensional Lorentzian manifold M


Displaying its nonredundant components:

In [19]:
Riem.display_comp(only_nonredundant=True)

Riem(g)^t_r,t,r = -2*m/(2*m*r^2 - r^3) 
Riem(g)^t_th,t,th = -m/r 
Riem(g)^t_ph,t,ph = -m*sin(th)^2/r 
Riem(g)^r_t,t,r = -2*(2*m^2 - m*r)/r^4 
Riem(g)^r_th,r,th = -m/r 
Riem(g)^r_ph,r,ph = -m*sin(th)^2/r 
Riem(g)^th_t,t,th = (2*m^2 - m*r)/r^4 
Riem(g)^th_r,r,th = -m/(2*m*r^2 - r^3) 
Riem(g)^th_ph,th,ph = 2*m*sin(th)^2/r 
Riem(g)^ph_t,t,ph = (2*m^2 - m*r)/r^4 
Riem(g)^ph_r,r,ph = -m/(2*m*r^2 - r^3) 
Riem(g)^ph_th,th,ph = -2*m/r

We can **lower and raise all the indices** of the components $R^\lambda_{\ \, \mu\nu\sigma}$ of the Riemann tensor, via the metric $g$ by the methods `down()` and `up()`.

In [20]:
Riemdown = Riem.down(g);
Riemup = Riem.up(g);

## Ricci tensor

We know that the Ricci tensor is computed via the Riemann curvature tensor: $R_{\mu\nu}={R^\lambda}_{\mu\lambda\nu}$. However, SageMath can give us directly the Ricci tensor from the metric $g$ with the method `g.ricci()`.

In [21]:
Ric = g.ricci()

In [22]:
Ric.display()

Ric(g) = 0

In [23]:
Ric[:]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

Let us check that the definition of the Ricci tensor via the contraction of the Riemann tensor and the one given by the SageMath method `g.ricci()` coincides. 

In [24]:
Ric == Riem.down(g)['_{abcd}']*ginv['^{ac}']

True

## Calculating the Scalar Curvature

It is computed by the contraction of theinverse metric and the Ricci tensor, i.e., $R = g^{\mu\nu} R_{\mu\nu}$.

In [25]:
ScalarCurvature=ginv['^{ab}']*Ric['_ab']; 
ScalarCurvature.display()

zero: M --> R
   (t, r, th, ph) |--> 0

## Einstein tensor

The Einstein tensor is found from the tensors already computed, $G_{\mu\nu} = R_{\mu\nu}-\frac{1}{2}g_{\mu\nu}R$.

In [26]:
EinsteinTensor = Ric-g*ScalarCurvature/2

In [27]:
EinsteinTensor[:]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

A zero matrix means that the vacuum Einstein equation is satisfied.

## Kretschmann scalar

The Kretschmann scalar is the "square" of the Riemann tensor defined by 
$$ K = R_{\lambda\mu\nu\sigma} R^{\lambda\mu\nu\sigma}$$
To compute it, we must first form the tensor fields whose components are $R_{\lambda\mu\nu\sigma}$ and 
$R^{\lambda\mu\nu\sigma}$. They are obtained by respectively lowering and raising the indices of the components $R^\lambda_{\ \, \mu\nu\sigma}$ of the Riemann tensor, via the metric $g$. These two operations are performed by the methods `down()` and `up()`. The contraction is performed by summation on repeated indices, using LaTeX notations:

In [28]:
K = Riem.down(g)['_{abcd}'] * Riem.up(g)['^{abcd}']
K

Scalar field on the 4-dimensional Lorentzian manifold M

In [29]:
K.display()

M --> R
(t, r, th, ph) |--> 48*m^2/r^6

In [30]:
K.expr()

48*m^2/r^6

## Levi-Civita Connection

The Levi-Civita Connection $\nabla$ associated with the metric $g$.

In [31]:
nab = g.connection() ; print(nab)

Levi-Civita connection nabla_g associated with the Lorentzian metric g on the 4-dimensional Lorentzian manifold M


We check the compatibility between metric and connection (that is, we must get $\nabla_g g=0$).

In [32]:
nab(g).display()

nabla_g(g) = 0

In [33]:
w = M.vector_field('w')

Compute the covariant derivative of the vector $w = (-t^2,r^2,0,0)$, $\nabla_\nu w^\nu$.

In [34]:
w[:] = [-t^2,r^2,0,0]

In [35]:
DW = (nab(w)['^a_b']*delta['_a^b'])
DW.expr()

4*r - 2*t

Check that $\nabla_\nu w^\nu = \partial_\nu w^\nu + w^\gamma {\Gamma^{\nu}}_{\gamma \nu}$.

In [36]:
sum([w[i].diff(i)+w[i]*sum([g.christoffel_symbols()[j,i,j] for j in M.irange()]) for i in M.irange()])    

4*r - 2*t